# <font color="red"> Project Introduction</font>
The aim of this project is to develop a forecasting model for a public transportation system. The transportation system under consideration handles a significant volume of passengers and operates on a regular schedule. Accurate forecasting of passenger demand is crucial for effective resource allocation, operational planning, and improving overall service efficiency.

##  <font color="blue"> Task Description </font>
The task at hand is to develop a robust and accurate forecasting model using time series analysis techniques. The model should be capable of capturing the underlying patterns, trends, and seasonality in the passenger demand data. The primary objective is to generate reliable forecasts for future time periods, enabling proactive decision-making and efficient resource management.

To achieve this task, the following steps are involved:

* Exploratory Analysis: Conduct a thorough exploratory analysis of the historical passenger demand data to identify patterns, trends, and seasonality. This analysis will provide insights into the underlying dynamics of the transportation system.

* Data Preprocessing: Preprocess the data by handling missing values, outliers, and any other data quality issues. Transform the data into a suitable format for time series analysis.

* Model Selection: Select an appropriate time series forecasting model that can effectively capture the patterns and seasonality present in the data. Consider models such as SARIMA (Seasonal Autoregressive Integrated Moving Average) that are well-suited for capturing complex time series dynamics.

* Model Training and Evaluation: Train the selected forecasting model using historical data. Evaluate the model's performance using appropriate evaluation metrics such as mean absolute error (MAE), mean absolute percentage error (MAPE), or root mean squared error (RMSE).

* Forecasting and Future Period Analysis: Generate forecasts for future time periods to estimate the expected passenger demand. Analyze the forecasted results to identify potential trends, seasonality, and patterns that can inform decision-making and resource allocation.

* Model Optimization and Validation: Fine-tune the model by optimizing hyperparameters, applying regularization techniques, and conducting model validation using validation data. This step helps ensure the model's robustness and generalization capabilities.

* Reporting and Visualization: Present the forecasting results through visualizations, plots, and interactive graphs. Communicate the findings and insights effectively, highlighting the model's performance, limitations, and recommendations for future improvements.

By successfully completing these tasks, the project aims to provide an accurate forecasting solution for the public transportation system, enabling stakeholders to make informed decisions and optimize resource allocation based on anticipated passenger demand.



#  <font color="red"> Importing Libraries and Loading Data </font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import plotly.express as px
import plotly.graph_objects as go
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("/kaggle/input/wileydataminingforbusinessanalyticsinr/bicup2006.csv")


#  <font color="red">  Exploratory Data Analaysis </font>

In [2]:
data.sample(5)

,DATE,TIME,DEMAND
1390,23-Mar-05,7:30,NaN
15,1-Mar-05,10:15,27.0
550,9-Mar-05,18:00,66.0
615,10-Mar-05,18:30,77.0
771,13-Mar-05,10:15,0.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    1512 non-null   object 
 1   TIME    1512 non-null   object 
 2   DEMAND  1323 non-null   float64
dtypes: float64(1), object(2)
memory usage: 35.6+ KB


In [4]:
data.describe()

,DEMAND
count,1323.000000
mean,25.872260
std,23.522793
min,0.000000
25%,4.000000
50%,23.000000
75%,40.000000
max,136.000000


In [5]:
fig = px.scatter(data, x='DATE', y='DEMAND', hover_data=['TIME'])
fig.show()


In [6]:
fig = px.histogram(data, x='DEMAND')
fig.show()


In [7]:
fig = px.box(data, y='DEMAND', x='DATE')
fig.show()


##  <font color="blue"> Identify outliers </font>

In [8]:
def detect_outliers(data, threshold=3):
    # Calculate z-score for each data point
    z_scores = (data - np.mean(data)) / np.std(data)
    
    # Identify outliers based on the z-score threshold
    outliers = np.abs(z_scores) > threshold
    
    return outliers

# Detect outliers in the data
outliers = detect_outliers(data['DEMAND'])

# Print the indices of the outlier data points
print("Outlier indices:")
data[outliers]

Outlier indices:


,DATE,TIME,DEMAND
50,1-Mar-05,19:00,98.0
425,7-Mar-05,18:15,100.0
426,7-Mar-05,18:30,115.0
427,7-Mar-05,18:45,136.0
428,7-Mar-05,19:00,100.0
489,8-Mar-05,18:30,98.0
866,14-Mar-05,18:15,113.0
867,14-Mar-05,18:30,108.0
868,14-Mar-05,18:45,116.0
869,14-Mar-05,19:00,98.0


# <font color="red">  Data Preprocessing </font>

##  <font color="blue"> Frequency of the Data </font>
In this step, the data is resampled to 15-minute intervals to determine the appropriate frequency for modeling. The original data, consisting of dates and times, is combined into a single 'DATE_TIME' column. The 'DATE_TIME' column is then set as the index of the data.

To achieve a consistent and manageable frequency, the data is resampled using the 'resample' function with a time interval of 15 minutes ('15T'). The resampling process calculates the mean value for each 15-minute interval, providing a representative value for the respective period.

By resampling the data to 15-minute intervals, we establish a uniform time series frequency that facilitates subsequent analysis and modeling tasks.

In [9]:
data['DATE_TIME'] = pd.to_datetime(data['DATE'] + ' ' + data['TIME'])
data.set_index('DATE_TIME', inplace=True)
data = data.resample('15T').mean()  # Resample to 15-minute intervals


###  <font color="green"> Seasonal Patterns of Demand in Public Transportation </font>
The plot helps identify recurring patterns, such as daily, weekly, monthly, or yearly fluctuations in demand. By observing these seasonal patterns, transportation authorities and planners can gain insights into peak periods, identify trends, and make informed decisions regarding resource allocation, scheduling, and capacity planning.

Overall, this visualization assists in understanding the seasonality of demand in public transportation, aiding in better management and optimization of resources to meet passenger needs efficiently.


In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['DEMAND'], mode='lines', name='Demand'))

years = pd.date_range(start=data.index[0], end=data.index[-1], freq='Y')
for year in years:
    fig.add_shape(type='line', x0=year, x1=year, y0=0, y1=1, line=dict(color='red', width=1, dash='dash'))

# Set x-axis and y-axis labels
fig.update_layout(xaxis_title='Date', yaxis_title='Demand')

# Set plot title
fig.update_layout(title='Seasonal Patterns')

# Show the interactive plot
fig.show()

##  <font color="blue"> Weekdays vs. Weekends </font>

In [11]:

data['Weekday'] = data.index.weekday
weekday_data = data[data['Weekday'] < 5]
weekend_data = data[data['Weekday'] >= 5]

###  <font color="green"> Public Transportation Demand Patterns by Weekday and Weekend</font>
The interactive plot showcases the patterns of public transportation demand by distinguishing between weekdays and weekends. The plot displays the number of passengers over time for both weekday and weekend periods. This visualization helps to identify and compare the demand patterns on different days of the week, providing insights into the variations in public transportation usage. The x-axis represents the date, while the y-axis represents the number of passengers. The plot allows for interactive exploration and analysis of the data.

In [12]:
fig = go.Figure()

# Add the weekday data plot
fig.add_trace(go.Scatter(x=weekday_data.index, y=weekday_data['DEMAND'],
                         mode='lines', name='Weekdays'))

# Add the weekend data plot
fig.add_trace(go.Scatter(x=weekend_data.index, y=weekend_data['DEMAND'],
                         mode='lines', name='Weekends'))

# Set the layout of the plot
fig.update_layout(
    title='Public Transportation Demand',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    height=600,
    width=900,
)

# Show the interactive plot
fig.show()


## <font color="blue">  Handling missing values </font>

In [13]:
# Handle missing values or unusual values in the dataset
data = data.dropna()  # Remove rows with missing values from the original DataFrame


##  <font color="blue"> Split the data into training and validation sets </font> 

In [14]:
training_data = data.iloc[:960]  # First 10 days (960 data points)
validation_data = data.iloc[960:]  # remainning days (363 data points)


# <font color="red">  Modelling </font> 

##  <font color="blue"> Regularized SARIMAX Model Fitting with Order  and Seasonal Order  </font>
In this step, we fit the SARIMAX model to the training data while incorporating regularization techniques. The order parameter is set to (1, 1, 1) for the non-seasonal component, indicating an autoregressive order of 1, differencing of order 1, and moving average order of 1. The seasonal_order parameter is set to (1, 1, 1, 96), indicating a seasonal autoregressive order of 1, seasonal differencing of order 1, seasonal moving average order of 1, and a seasonal period of 96 (assuming 15-minute intervals).

To mitigate overfitting and reduce model complexity, a regularization parameter of 0.5 is applied. The enforce_stationarity and enforce_invertibility parameters are set to True to ensure that the model remains stationary and invertible, respectively.

The SARIMAX model is then fitted to the training data using the specified parameters, taking into account regularization. This step helps improve the model's generalization and reduces the risk of overfitting by controlling the model's complexity and preventing extreme parameter estimates.

In [15]:
# Fit the SARIMAX model to the training data with regularization
order = (1, 1, 1)  
seasonal_order = (1, 1, 1, 96)  
regularization_param = 0.5  # Regularization parameter 
model = SARIMAX(training_data['DEMAND'], order=order, seasonal_order=seasonal_order,
                enforce_stationarity=True, enforce_invertibility=True,
                regularization=regularization_param)
fitted_model = model.fit()


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.34727D+00    |proj g|=  3.97047D-02


 This problem is unconstrained.



At iterate    5    f=  3.30115D+00    |proj g|=  2.29726D-02

At iterate   10    f=  3.28154D+00    |proj g|=  2.20635D-03

At iterate   15    f=  3.28087D+00    |proj g|=  3.07107D-03

At iterate   20    f=  3.28085D+00    |proj g|=  1.27075D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     24     26      1     0     0   1.242D-05   3.281D+00
  F =   3.2808407144524572     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


##  <font color="blue">  Generate forecasts for the training and validation periods </font>

In [16]:
training_forecasts = fitted_model.predict(start=0, end=len(training_data)-1)
validation_forecasts = fitted_model.predict(start=len(training_data), end=len(training_data)+len(validation_data)-1)


##  <font color="blue"> Calculate MAPE and MAE for training and validation periods </font>
The code snippet calculates the Mean Absolute Percentage Error (MAPE) and Mean Absolute Error (MAE) for the training and validation periods in a time series forecasting task. These metrics provide insights into the accuracy and performance of the SARIMA model.

The MAPE measures the average percentage difference between the actual demand values and the corresponding forecasted values. It indicates the relative magnitude of the forecast errors. A lower MAPE value signifies a better fit of the model to the training and validation data.

The MAE represents the average absolute difference between the actual and forecasted demand values. It provides a measure of the magnitude of the forecast errors, regardless of their direction. Similar to MAPE, a lower MAE indicates a more accurate model fit.

By evaluating the MAPE and MAE for both the training and validation periods, it is possible to assess the model's performance on both seen and unseen data. This evaluation helps determine if the model is overfitting or underfitting the training data and provides insights into its generalization capability.

In [17]:
# Step 4: Calculate MAPE and MAE for training and validation periods
training_mape = mean_absolute_percentage_error(training_data['DEMAND'], training_forecasts)
training_mae = mean_absolute_error(training_data['DEMAND'], training_forecasts)

validation_mape = mean_absolute_percentage_error(validation_data['DEMAND'], validation_forecasts)
validation_mae = mean_absolute_error(validation_data['DEMAND'], validation_forecasts)


In [18]:
# Generate forecasts for the future period
future_forecasts = fitted_model.forecast(steps=336)  # 336 steps for 3 days (15-minute intervals)

#  Combine training and validation data for the final model
combined_data = pd.concat([training_data, validation_data])


##  <font color="blue"> Forecasted Demand vs. Actual Demand: Final Model Fit </font>

In [19]:
import plotly.graph_objects as go

# Create the figure and add traces for actual and forecasted data
fig = go.Figure()

# Add trace for actual data
fig.add_trace(go.Scatter(x=combined_data.index, y=combined_data['DEMAND'], mode='lines', name='Actual'))

# Add trace for forecasted data
fig.add_trace(go.Scatter(x=combined_data.index, y=np.concatenate([training_forecasts, validation_forecasts]), mode='lines', name='Forecast'))

# Set x-axis and y-axis labels
fig.update_layout(xaxis_title='Date', yaxis_title='Demand')

# Set plot title
fig.update_layout(title='Final Model Fit')

# Add legend
fig.update_layout(legend=dict(x=0, y=1, traceorder="normal", font=dict(size=10)))

# Show the interactive plot
fig.show()


###  <font color="green"> Notice of Overfitting </font>
The provided plot suggests the presence of overfitting in the model. Overfitting occurs when the model performs exceptionally well on the training data but fails to generalize to unseen data. This issue can lead to inaccurate predictions and unreliable forecasts. It is important to address overfitting to ensure the model's effectiveness and reliability.
###  <font color="orange"> Reasons for Overfitting : </font>
* Insufficient Training Data
* using wronf values for Order and Seasonal Order 

###  <font color="orange"> Solutions to Address Overfitting : </font>
* Regularization Techniques 'used'
* Tune the hyperparameters carefully using appropriate validation techniques.' take very long time'

# <font color="red"> Task solution </font> 

In [20]:
# Print the summary
print("TASK_1. Method/Combination: SARIMA")
print("*"*15)
print("TASK_2. SARIMA, which stands for Seasonal Autoregressive Integrated Moving Average, is a popular time series analysis model used for forecasting. It is an extension of the ARIMA model that incorporates seasonal patterns into the analysis. SARIMA models are effective for analyzing and forecasting time series data that exhibit both non-seasonal and seasonal trends.")
print("*"*15)
print("TASK_3. Estimated Equations: SARIMA(" + str(order) + ")x(" + str(seasonal_order) + ")")
print("*"*15)
print(f"TASK_4. MAPE - Training: {training_mape:.4f}")
print(f"   MAE - Training: {training_mae:.4f}")
print(f"   MAPE - Validation: {validation_mape:.4f}")
print(f"   MAE - Validation: {validation_mae:.4f}")
print("*"*15)
print("TASK_5. Forecasts for Future Period (22-24 Mars):")
fdate='2005-03-21 23:45:00'
fdate=pd.to_datetime(fdate, format='%Y-%m-%d %H:%M:%S')
for i, forecast in enumerate(future_forecasts):
    date =fdate + pd.Timedelta(minutes=15*(i+1))
    print(f"   {date}: {forecast:.2f}")

TASK_1. Method/Combination: SARIMA
***************
TASK_2. SARIMA, which stands for Seasonal Autoregressive Integrated Moving Average, is a popular time series analysis model used for forecasting. It is an extension of the ARIMA model that incorporates seasonal patterns into the analysis. SARIMA models are effective for analyzing and forecasting time series data that exhibit both non-seasonal and seasonal trends.
***************
TASK_3. Estimated Equations: SARIMA((1, 1, 1))x((1, 1, 1, 96))
***************
TASK_4. MAPE - Training: 975091296765194.1250
   MAE - Training: 6.8687
   MAPE - Validation: 9041468711541630.0000
   MAE - Validation: 19.5094
***************
TASK_5. Forecasts for Future Period (22-24 Mars):
   2005-03-22 00:00:00: 18.79
   2005-03-22 00:15:00: 18.30
   2005-03-22 00:30:00: 24.81
   2005-03-22 00:45:00: 24.10
   2005-03-22 01:00:00: 26.88
   2005-03-22 01:15:00: 25.14
   2005-03-22 01:30:00: 23.53
   2005-03-22 01:45:00: 24.68
   2005-03-22 02:00:00: 27.06
   2005

In [21]:
print("TASK_6. A single chart showing the fit of the final version of the model to theentire period (including training, validation, and future). :")

import plotly.graph_objects as go

# Create the figure and add traces for actual and forecasted data
fig = go.Figure()

# Add trace for actual data
fig.add_trace(go.Scatter(x=combined_data.index, y=combined_data['DEMAND'], mode='lines', name='Actual'))

fig.add_trace(go.Scatter(x=combined_data.index, y=np.concatenate([training_forecasts, validation_forecasts]), mode='lines', name='Forecast'))

# Add trace for forecasted data
forecast_dates = [fdate + pd.Timedelta(minutes=15*i) for i in range(len(future_forecasts))]
fig.add_trace(go.Scatter(x=forecast_dates, y=future_forecasts, mode='lines', name='Futere'))

# Set x-axis and y-axis labels
fig.update_layout(xaxis_title='Date', yaxis_title='Demand')

# Set plot title
fig.update_layout(title='Forecasts for Future Period')

# Add legend
fig.update_layout(legend=dict(x=0, y=1, traceorder="normal", font=dict(size=10)))

# Show the interactive plot
fig.show()


TASK_6. A single chart showing the fit of the final version of the model to theentire period (including training, validation, and future). :


###  <font color="orange"> TASK_7. Discuss the limitations and challenges encountered during the forecasting process and propose potential strategies to overcome them : </font>
* Limited or Incomplete Data
* Limited Time
* Data Quality and Accuracy
* Seasonality and Trend Changes


###  <font color="orange"> TASK_8 Explain how your forecasting method can be applied to other public transportation systems and the potential benefits it can provide: </font>
The forecasting method we discussed, which incorporates time series analysis using SARIMAX models, can be applied to other public transportation systems with similar data characteristics. Here are the potential benefits it can provide:

* Accurate Demand Forecasting
* Improved Operational Efficiency
* Integration with Mobility Services
* Demand-Responsive Strategies